In [ ]:
import pandas as pd
import numpy as np

In [ ]:
with open('dialog_acts.dat', 'r') as f:
    data = f.readlines()
    data = list(map(lambda x: [x[0:x.index(" ")], x[x.index(" ")+1:-1]], data))

In [ ]:
df = pd.DataFrame(np.array(data), columns = ['label', 'text'])

In [ ]:
df.head()

,label,text
0,inform,im looking for a moderately priced restaurant ...
1,inform,any part of town
2,inform,bistro food
3,confirm,is there a moderately priced restaurant that s...
4,affirm,yes


In [ ]:
from sklearn.model_selection import train_test_split

# Pre-Processing

In [ ]:
df.head()

,label,text
0,inform,im looking for a moderately priced restaurant ...
1,inform,any part of town
2,inform,bistro food
3,confirm,is there a moderately priced restaurant that s...
4,affirm,yes


In [ ]:
df['text'].head()

0    im looking for a moderately priced restaurant ...
1                                     any part of town
2                                          bistro food
3    is there a moderately priced restaurant that s...
4                                                  yes
Name: text, dtype: object

In [ ]:
# if text contains "noise" or "tv_noise", remove
# if that makes the column empty, remove the row

for index, row in df.iterrows():
    print(row['c1'], row['c2'])

#bot

In [ ]:
def conversation():
  finished = False
  while not finished:
    print(f'Hello. What do you want?')

    
    user = input().lower()
    #get keyword from user input
    #assign label
    
    
    if 'bye' in user:
      finished = True

In [ ]:
conversation()

Hello. What do you want?
bye


#split

In [ ]:
from sklearn.model_selection import train_test_split
msg_train, msg_test, label_train, label_test = train_test_split(df['text'], df['label'], test_size=0.15, random_state=10)